In [3]:
import pandas as pd

In [5]:
def validate_train_data(file_path):
    """
    Функція для перевірки коректності підготовлених даних для моделі BERT.
    :param file_path: str, шлях до файлу train_data.csv
    :return: None (друкує результати перевірки)
    """
    try:
        # Завантаження даних
        data = pd.read_csv(file_path)

        print("\nПеревірка структури файлу csv")
        print("--------------------------------------------------")

        # Перевірка, що файл завантажено коректно
        if data.empty:
            raise ValueError("Файл csv порожній.")

        # Перевірка обов'язкових колонок
        required_columns = ["input_ids", "attention_masks", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
        missing_columns = [col for col in required_columns if col not in data.columns]
        if missing_columns:
            raise ValueError(f"Відсутні обов'язкові колонки: {', '.join(missing_columns)}")

        print("✔ Усі обов'язкові колонки присутні.")

        # Перевірка розміру даних
        num_rows = len(data)
        print(f"✔ Файл містить {num_rows} рядків.")

        if num_rows < 100:
            print("⚠ Попередження: Мала кількість рядків. Можливо, дані неповні.")

        # Перевірка коректності формату input_ids і attention_masks
        for col in ["input_ids", "attention_masks"]:
            if not data[col].apply(lambda x: isinstance(x, str)).all():
                raise ValueError(f"Колонка {col} містить некоректні дані. Очікувався строковий формат JSON.")

        print("✔ Формат колонок input_ids і attention_masks коректний.")

        # Перевірка міток (toxic, severe_toxic, obscene, threat, insult, identity_hate)
        for label in required_columns[2:]:
            unique_values = data[label].unique()
            if not set(unique_values).issubset({0, 1}):
                raise ValueError(f"Колонка {label} містить некоректні значення: {unique_values}. Очікувались тільки 0 або 1.")

        print("✔ Формат міток коректний.")

        print("\nФайл csv успішно пройшов усі перевірки!")

    except Exception as e:
        print(f"Помилка перевірки даних: {e}")

In [6]:
validate_train_data("train_data.csv")


Перевірка структури файлу csv
--------------------------------------------------
✔ Усі обов'язкові колонки присутні.
✔ Файл містить 159571 рядків.
✔ Формат колонок input_ids і attention_masks коректний.
✔ Формат міток коректний.

Файл csv успішно пройшов усі перевірки!


In [8]:
validate_train_data("test_data.csv")


Перевірка структури файлу csv
--------------------------------------------------
✔ Усі обов'язкові колонки присутні.
✔ Файл містить 63978 рядків.
✔ Формат колонок input_ids і attention_masks коректний.
✔ Формат міток коректний.

Файл csv успішно пройшов усі перевірки!


In [13]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [14]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from transformers import TFBertModel
from tensorflow.keras.layers import Input, Layer, Dense
from tensorflow.keras.models import Model

In [15]:
# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    test[column] = test[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(test['input_ids'].values)
attention_mask = np.stack(test['attention_masks'].values)
labels = np.array(test[LABEL_COLUMNS].values)
labels = labels.astype('float32')

# Розділення на тренувальну та тестову вибірки

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

# Перевірка форматів даних 
print("input_ids dtype:", input_ids.dtype)
print("attention_mask dtype:", attention_mask.dtype)
print("labels dtype:", labels.dtype)

# Перевірка довжини даних
print("Lengths match:", len(input_ids) == len(attention_mask) == len(labels))

input_ids dtype: int32
attention_mask dtype: int32
labels dtype: float32
Lengths match: True


In [16]:
# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    train[column] = train[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(train['input_ids'].values)
attention_mask = np.stack(train['attention_masks'].values)
labels = np.array(train[LABEL_COLUMNS].values)
labels = labels.astype('float32')

# Розділення на тренувальну та тестову вибірки

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

# Перевірка форматів даних 
print("input_ids dtype:", input_ids.dtype)
print("attention_mask dtype:", attention_mask.dtype)
print("labels dtype:", labels.dtype)

# Перевірка довжини даних
print("Lengths match:", len(input_ids) == len(attention_mask) == len(labels))

input_ids dtype: int32
attention_mask dtype: int32
labels dtype: float32
Lengths match: True
